# Motivation and Overview of Data:

## Project Purpose:

There is alot of unknown flexability when it comes to recipes.  When you open up a recipe in a book, it is hard to know what leaneancy you have.

We want to compare recipies of different types to find trends in their nutritional value.  We hope that by creating a 

## Questions to Answer:


## Background knowledge/resources: 
Talk about what has been done, and what this will cover that is different.

this is a secret edit

## Data overview:
Talk about what the data looks like before we talk about how we got it (don't walk them through the cleaning process; walk them through what they will see, then talk about how we got it/cleaned it)

# Data Collection and Cleaning:

## Collection Procedure:
talk about how data is used at each step:

The recipe website divided its recipies into categories. Some examples of these are: Chili, Lamb, Potatoes, and even Christmas recipes!
The code to scrape this website divides into 4 functions.

- first one gets the links on the main site
- Then we separate them into categories
- We then collect the recipes
- And we parse through them

![title](example_recipe.png)

## Collecting ingredients

Now that we have the individual ingredients we can start collecting nutritional data. The site that we used is okay with us using their data as long as we are not going to sell it.

The scaper we built for this task searches each ingredient on nutritionvalue.org and takes the first available 3 links. The reason for this is that the top link might not be what we are looking for but, since they are ordered by relevance, it is highly likely to be on the top three.

Example of query with Oats:
![title](nutrition_value_query.png)

These links are collected in a dictionary with the ingredient as its respective value. The code is able to handle common errors like "no results", "less than 3 links", and "can't find search bar.

code:

In [ ]:
## CODE: Scraper 1
def nutrition_website(ingredients):
    """Use Selenium to enter the given search query into the search bar of
    nutrion website and gets links to scrape data

    Returns:
        (dictonary): urls .
    """
    #initialize variables and chrome
    ingredients_dictionary = {}
    browser = webdriver.Chrome()
    browser.get("https://www.nutritionvalue.org/")
    num_links = 3
    try:
        for i in ingredients:
            try:
                #navigate
                search_bar = browser.find_element_by_name('food_query')
                search_bar.clear()
                search_bar.send_keys(Keys.CONTROL + "a")
                search_bar.send_keys(Keys.DELETE)
                search_bar.send_keys(i)

                search_bar.send_keys(Keys.RETURN)
                
                words = i.split()
                x = ""
                for n,w in enumerate(words):
                    if n == 0:
                        x += ".*(?<!food_query=)"+w+".*|"
                    else:
                        x += ".*(?<!\+)"+w+".*|"
                x = x[:-1]
                find = re.compile(x,re.IGNORECASE)
            
                
                # wait for page to load
                time.sleep(2)
                currentURL = browser.current_url
                if "food_query" in currentURL:
                    
                    links = browser.find_elements_by_tag_name('a')
                    links = [link.get_attribute("href") for link in links if isinstance(link.get_attribute("href"),str)]
                    urls = [link for link in links if len(find.findall(link)) > 0]
                    if len(urls) >num_links:
                        ingredients_dictionary[i] = urls[:num_links]
                    elif len(urls) == 0:
                        ingredients_dictionary[i] = None
                    else:
                        ingredients_dictionary[i] = urls
                else:
                    ingredients_dictionary[i] = [currentURL]
                

            except NoSuchElementException:
                print("could not find the search bar!")
                print(i)
                return ingredients_dictionary
    # close window
    finally:
        browser.close()
    # list with all the links
    return ingredients_dictionary


#### Nutrition Values

Now we are ready to get the nutritional value.

Each ingredient link contains tables of nutritonal properties and their quantities. Here is an example of the content of the link related to "Oats".

![](nutrition_values_oats.png)

The scaper will go through the dictionary of links and collect the name, serving size, macronutries (Protein, fat, carbohydrates), micronutrients (vitamins and minerals), and any other fact on the tables available.

The code prints out any links that cause an error to help minimize the number of missing ingredients. There were a few links with error but we inspected them and made sure we weren't losing any important information. They turned out to be links that were not related to food but rather other parts of the website that were at times included in our query. 

In order to not abuse the websites information we also added an argument that skips the search if it has already been pulled.

code:

In [ ]:
# code for scraper
def nutrition_value(dictionary,set_of_links = set()):
    """Takes in a dictionary with ingredients as keys
    look through the websites and scrape the nutritional value"""
    error_items =[]
    df_d = dict()
    browser = webdriver.Chrome()
    try:
        for k,v in zip(dictionary.keys(),dictionary.values()):
            if v is None:
                df_d[k] = {}
            else:    
                for l in v:
                    try:
                        if l is None:
                            print(f"No Website for: {k}")
                        elif l in set_of_links:
                            print(f"Duplicates for {k}")
                        else:
                            browser.get(l)
                            time.sleep(5)
                            # name of ingredient
                            name = browser.find_elements_by_tag_name('h1')[0]
                            name = name.text

                            #setting up nutritional values
                            nut = dict()
                            c = "tbody"
                            tables = browser.find_elements_by_tag_name(c)
                            #### For essentials [4]
                            ser_cal = tables[4].text.split('\n')
                            # Serving Size
                            nut[ser_cal[1][:12]] = ser_cal[1][13:]
                            #Calories
                            nut[ser_cal[3][:8]] = ser_cal[3][9:]

                            #### For all others [7-13]
                            n_v = re.compile('\s*(.*)\s([0-9]+\.[0-9]+\s\w+)')
                            for i in range(7,14):
                                nutrient_value = [n_v.findall(t) for t in tables[i].text.split('\n') if len(t) >0]
                                for t in nutrient_value:
                                    if len(t)>0:
                                        nut[t[0][0]] = t[0][1]

                            df_d[name] = nut
                            set_of_links.add(l)
                        
                    except IndexError as e:
                        error_items.append(k)
                        print(f"ingredient:{k}, error: {e}, link: {l}")
                    except:
                        error_items.append(k)
                        print(f"ingredient:{k}, error: IDK, link: {l}")
            
    finally:
        browser.close()
    # list with all the links
    df = pd.DataFrame.from_dict(df_d,'index')
    return df, error_items, set_of_links

#### Cleaning Nutritional Data

Once we collected the data we analyzed it for errors. There were two columns that had only one value out of all the ingredients - the number "18" and "adjusted Protein". For this reason we dropped those columns. 

The rest of the data was cleaned by making all values floats, converting values to grams (g) and making serving sizes be 1g for all ingredients.

Vitamin A and D were a special case because they were measured in International Units (IU) so we converted them to grams

Lastly, we engineered a columns for all minerals and vitamins for future investigation of trends in nutrition - Whether you can get all the nutrition you need from certain foods or recipes.

This is our nutrtional_value dataframe

Code:

In [ ]:
# floats for calories
df = ds
df["Calories"] = df['Calories'].astype('float')/100
df = df.drop(columns = ["18","Adjusted Protein"]) # contains singleton

for j,c in enumerate(list(df.columns)):
    if c ==  "Calories":     
        pass
    else:    
        n = re.compile(r"(^\d*\.?\d*)\s(\w+)")     # float values
        
        ### changing Na for 0's, to go back change 0 to np.nan
        num = np.array([float(n.findall(i)[0][0]) if isinstance(i,str) else 0 for i in df[c].values])
        mes = np.array([n.findall(i)[0][1] if isinstance(i,str) else 'g' for i in df[c].values])

        # messurements
        mask_mg = (mes == 'mg')/10
        mask_mcg = (mes == "mcg")/10000
        mask_g = (mes == "g")*.01
        mask = mask_mg + mask_mcg +mask_g
        mask += (mask==0)*-1

        if sum(mask < 0) >1 :
            df[c] = num/100
        else:
            df[c] = num*mask

df["Vitamin A"] *= 0.6/1000000
df["Vitamin D"] *= 0.025/1000000

vitamins=['Choline','Niacin','Pantothenic acid','Riboflavin','Thiamin',
          'Vitamin A','Vitamin B12','Vitamin B6','Vitamin C','Vitamin D','Vitamin E','Vitamin K']
minerals = ['Calcium, Ca','Copper, Cu','Iron, Fe',    'Magnesium, Mg',
            'Manganese, Mn','Phosphorus, P','Potassium, K','Selenium, Se','Sodium, Na','Zinc, Zn']
# append new features
df['Vitamins'] = df[vitamins].sum(axis=1)
df['Minerals'] = df[minerals].sum(axis=1)
df.head()

Now that both data sets are clean and contain featured columns, we begin the merging process to produce a giant sparse dataframe where the rows are recipies and the columns are ingredient's (ing) nutritional properties for all ingredients. example:  

|recipe|ing1_calories |ing1_protein |ing1_vitamins |... | ing1_minerals| ing2_calores| ...|total_calories|...|total_minerals|
|-|-|-|-|-|-|-|-|-|-|-|


In the transition to one data frame we encountered one of our biggest challenges thus far. We need to sort through the quantities of each ingredient and convert them to grams. Unfortunately, we have not found a comprehensive list of food densities online, and the measurements are not consistent on the recipe website. We have created a temporal solution that converts abstract measurements into the equivalent in grams for the most used ingredient and we map words that mean the same things to a uniform name (ex: tbs, tablesp, TBS -> tablespoon, See appendix for code).

Small Example of unit converter:

In [ ]:
# u contains uniform words
conv = {c:[] for c in set(u.values())}
# g/ units
conv['gram'] = 1
conv['liter'] = 1000
conv["gallon"] = 0.00026417205
conv['lbs'] = 453.592
conv['milliliter'] = 1
conv['oz']= 28.34
conv['kilogram'] = 1000
conv['tablespoon'] = 14.3

We are continually searching for a solution to the conversion problem. Nevertheless, We proceed with the analysis so that when an improvement to this method is found we can get better results. 

There are multiple steps to build what we are calling "Hefty_df". For each recipe we find the ingredients it requires and look up the units and quantities. For the quantities we built a function that takes in strings of the form "1/2 ,5 3/2, None" and returns its float equivalent or 1 for None (it represents "whole"). This ingredient is found in our ingredients/nutrional value dataframe by the Levenshtein distance which compares the similarity of words by comparing how many edits are needed to change one word into the other. For example by adding or dropping a letter.  We then multiply the quantity, unit (using our unit converter), and the nutritional value of the ingredient and place them into hefty_df. Lastly, we sum the values of each property and enter it into the "total_(respective property)" and repeat the process (*see apendix for code).

# INSERT PICTURE OF HEAD OF HEFTY

The reason we built this sparse dataframe is for the intends and purposes of next semester. We use the columns with the total nutritional values columns and the ingredients/nutritional value dataframe for our analyzis.

# Data Visualization and Analysis

- PROTEIN

- FAT

There is an important distinction between the different fat values printed on a produce. Our body needs the good fats ("Polyunsaturated fatty acids" and "Monounsaturated fatty acids") and could go without the bad ones ("Saturated fatty acids" and "Trans fats"). We wish to find out how recipe styles vary in their "good fat to bad fat" ratio. It is important to note that these are not true for just one ingredient (ex: fish, chicke, asparagus,etc.) or a single recipe, rather and combination of many recipies of that kind.

First we will look at the recipes with high good/bad ratio:

![](goodbad1.png)

These are the recipies that would be recommended if someone put the constraint to have more good fats rather than bad. These results look resonable since they all tend to have leaner meet and/or recipes that are considered "healthier".

We chose to remove the following categories From the graph above:
- Alchoholic beverage - Most of the ingredients are not well represented since they are too specific and were put in the "other" ingredients pile
- Jam - Likely there are low leves of fat which would make the ratio extreme depending on one or two recipes
- Dog Buiscuits - The reader can know that they are good for their dog 

Likewise, you might want to know which recipies to abstain from making/eating to stay healthy. The following figure shows the lowest ratios of good to bad fat. 

![](goodbad2.png)

Once again we have values that we expect like cake, pasta, and breakfast foods. Asparagus is on there because it is often cooked with butter, cheese or oils. 

- Vitamins and Minerals

Last question we would like to address is whether there is any correlation between protein, vitamins, and minerals in our list of ingredients. If so then we know that choosing one high nutritious ingredient will provide a balance more balance to your diet.

![](corr_m.png)

The highest correlation is between protein and Minerals which makes sense since there is iron in meat, and the lowest is between vitamins and minerals - Probably the reason they are sold seperately as supplements.

Code Quality and Robustness:

Data Visualization and Analysis:

# Apendix

### Code for hefty DataFrame

In [ ]:
# changing units by hand (only first time seen)
u = dict()
for i,rec in enumerate(neal_df[mask].values):
    for j,mes in enumerate(rec):
        if mes in u:
            pass
        else:
            print(neal_df[mask].columns[j])
            print(mes)
            u[mes] = input()
            print()

In [ ]:
# Unit Conversion dictionary
# g/ units
conv['gram'] = 1
conv['liter'] = 1000
conv["gallon"] = 0.00026417205
conv['lbs'] = 453.592
conv['milliliter'] = 1
conv['oz']= 28.34
conv['kilogram'] = 1000
conv['tablespoon'] = 14.3
conv['teaspoon'] = 4.77
conv["cup"] = 201
conv["ear"] = 92
conv['clove'] = 7
conv['pinch'] = 0.36
conv['quart'] = 946.353
conv['pint'] = 473.176
conv['envelope'] = 7.085
conv['None'] = 0
conv['dash'] = 0.72
conv['head'] = 539
conv['stick'] = 113
conv['package'] = 7.085
conv['small'] = 75
conv['medium'] = 150
conv['large'] = 225
conv['stalk'] = 50
conv['strip'] = 10
conv['square'] = 13
conv['square'] = 56.7
conv['box'] = 382.59
conv['whole'] = 100
conv['bag'] = 453.59
conv['sprig'] = 30
conv['bulb'] = 30
conv['slice'] = 5
conv['bunch'] = 120
conv['part'] = 1
conv['cube'] = 57

In [9]:
# gets mask for ingredients
col = list(neal_df.columns) 
mask_class = np.array([i for i in col if i[:6] == "class_"])

In [10]:
# create the columns for Hefty
hefty_columns = [nut_element+"_"+category[6:] for category in mask_class for nut_element in df.columns]
hefty_columns += ["Total "+ nut_element for nut_element in df.columns]

In [ ]:
# Turning strings to floats
def st_to_fl(s):
    try:
        # This implied "Whole"
        if s is None:
            return 1
        # other wise make it float
        return float(s)
    except ValueError:
        #if error do this
        return float(sum(Fraction(c) for c in s.split()))

In [ ]:
# building hefty
hefty_df = pd.DataFrame(columns=hefty_columns)
num_recipies = len(list(neal_df.index))

# for each recipe
for num, recipe in enumerate(neal_df.index):
    # get all the entries and get the values that are not None
    r = neal_df.iloc[recipe]
    r_contents = r[mask_class].values != None # mask
    
    #get the values where it wasn't none and inicialize the row
    ingredients = r[mask_class][r_contents].values
    hefty_df.loc[len(hefty_df)] = 0
    
    #progress bar
    if num% 200 == 0:
        print(f"{num/num_recipies}%")
        
    # for each ingredient in the recipe
    for ing in ingredients:
        # change the quantity to float and convert units to grams
        quantity = st_to_fl(r['quant_' + ing])
        unit_factor = conv[r['unit_' + ing]]
        conversion = quantity*unit_factor  
        
        #find the most similar ingredient in the nutrition data frame
        most_similar_ing = process.extractOne(ing,df.index)[0]
        
        # match the columns of hefty with the ingredient
        col = list(hefty_df.columns) 
        hefty_mask = np.array([i for i in col if i[-(len(ing)+1):] == "_"+ing])
        
        # for each column with the ingredient mutiply nutrition value by conversion
        # and add to total column
        for m,i,c in zip(hefty_mask, df.loc[most_similar_ing].values,df.columns):
            value = i*conversion
            hefty_df.loc[num][m] = value
            hefty_df.loc[num]["Total " + c] += value
